In [4]:
from utils.utils import gait_dataloader
from utils.utils import init_model,  patients_idx_to_cycles_idx, find_cycles_idx_by_patient_idx
from sklearn.model_selection import train_test_split

from utils.utils import init_model,accuracy_end_to_end, GridSearchCV

import numpy as np

from sklearn.svm import SVC

from sklearn.metrics import f1_score

from sklearn.metrics import accuracy_score

from sklearn.metrics import confusion_matrix

from captum.attr import visualization as viz
from captum.attr import Lime, LimeBase
from captum._utils.models.linear_model import SkLearnLinearRegression, SkLearnLasso
from captum.attr._core.lime import get_exp_kernel_similarity_function

def load_dataset_v1(dir_dataset,channel_first=True,flatten=True,shuffle=False,random_state=0):
    index = [0,1,2,3,4,5,6,7,8,9,14,15,16,17,18,19,20,21,22,23,24,29]
    
    npzfile = np.load(dir_dataset + 'RightPC.npz')
    
    x_train_all = npzfile['Input']
    y_train = npzfile['Output']
    y_train = np.ravel(y_train)
    x_train = x_train_all[:,:,index]
    if channel_first:
        x_train = x_train.transpose(0,2,1)
    
    npzfile = np.load(dir_dataset + 'LeftPC.npz')
    x_val_all = npzfile['Input']
    y_val = npzfile['Output']
    y_val = np.ravel(y_val)
    x_val = x_val_all[:,:,index]
    if channel_first:
        x_val = x_val.transpose(0,2,1)
    
    npzfile = np.load(dir_dataset + 'TD.npz')
    x_test_all = npzfile['Input']
    y_test = npzfile['Output']
    y_test = np.ravel(y_test)
    x_test = x_test_all[:,:,index]
    if channel_first:
        x_test = x_test.transpose(0,2,1)
    
    if flatten == True:
        x_train = x_train.reshape([x_train.shape[0],-1])
        x_val = x_val.reshape([x_val.shape[0],-1])
        x_test = x_test.reshape([x_test.shape[0],-1])
    
    nb_classes = len(np.unique(np.concatenate((y_train, y_val, y_test), axis=0)))
    
    return x_train, y_train ,x_val, y_val, x_test, y_test, nb_classes

dataset = "archives/AQM/TDvsPC_new_no_shuffle/"

x_R, y_R ,x_L , y_L, x_TD, y_TD, nb_classes = load_dataset_v1(dataset,channel_first=True,flatten=False,shuffle=False,random_state=0)

X_d = np.concatenate([x_R,x_L,x_TD])
# X_d = X_d.reshape([len(X_d),-1])
y_d = np.concatenate([y_R,y_L,y_TD])
import  pandas as pd

R_data = pd.read_csv('DataPC - Right.csv', header=None)
R_idx = R_data[2].to_numpy()


L_data = pd.read_csv('DataPC - Left.csv', header=None)
L_idx = L_data[2].to_numpy()
L_idx = L_idx + R_idx[-1]

TD_data = pd.read_csv('DataTD - By Subject.csv', header=None)
TD_idx = TD_data[2].to_numpy()
TD_idx = TD_idx + L_idx[-1]

cycle_end_idx = np.concatenate([R_idx,L_idx,TD_idx])
cycle_end_idx

patient_index_range = np.arange(len(R_idx)+len(L_idx)+len(TD_idx))
patient_class = np.concatenate([np.ones(len(R_idx)+len(L_idx)), np.zeros(len(TD_idx))])

x_train, x_test, y_train, y_test = train_test_split(patient_index_range, patient_class, test_size=0.4, stratify=patient_class,random_state=0)
x_val, x_test, y_val, y_test = train_test_split(x_test,y_test, test_size=0.75, random_state=0, stratify=y_test)
print(np.unique(y_train,return_counts=True))
print(np.unique(y_val,return_counts=True))
print(np.unique(y_test,return_counts=True))



classifer = init_model(model_name='ResNet').load('new_train_lrR_ET_TDvsCPu_full_5_original_weight_ResNet/model-epoch=04-val_loss=0.96-val_accuracy=0.92.ckpt')
classifer.predict(X_d[:10])

/tmp/ipykernel_575841/2820489161.py:68: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  R_data = pd.read_csv('DataPC - Right.csv', header=None)
/tmp/ipykernel_575841/2820489161.py:72: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  L_data = pd.read_csv('DataPC - Left.csv', header=None)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


(array([0., 1.]), array([35, 89]))
(array([0., 1.]), array([ 6, 14]))
(array([0., 1.]), array([18, 45]))
Predicting DataLoader 0: 100%|████████████████████| 1/1 [00:00<00:00, 56.61it/s]

/home/zhengyang/anaconda3/envs/env_gait3/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([1, 1, 0, 1, 1, 0, 1, 1, 1, 1])

In [6]:
type(X_d[:10])

numpy.ndarray